In [1]:
import streamm

In [2]:
methane = streamm.Buildingblock('methane')

In [3]:
C = streamm.Particle(symbol='C')
H = streamm.Particle(symbol='H')

In [4]:
methane.add_partpos(C,[0.0,0.0,0.0])
methane.add_partpos(H,[0.69,0.69,0.69])
methane.add_partpos(H,[-0.69,-0.69,0.69])
methane.add_partpos(H,[-0.69,0.69,-0.69])
methane.add_partpos(H,[0.69,-0.69,-0.69])

In [5]:
methane.bonded_nblist = methane.guess_nblist(0,radii_buffer=1.25)

In [6]:
for p_index,particle_i in methane.particles.iteritems():
    print p_index,particle_i,methane.positions[p_index],methane.bonded_nblist.calc_nnab(p_index),methane.unit_conf['length']

0 atom[0] C (C) [ 0.  0.  0.] 4 ang
1 atom[1] H (H) [ 0.69  0.69  0.69] 1 ang
2 atom[2] H (H) [-0.69 -0.69  0.69] 1 ang
3 atom[3] H (H) [-0.69  0.69 -0.69] 1 ang
4 atom[4] H (H) [ 0.69 -0.69 -0.69] 1 ang


In [7]:
methane.bonded_bonds()
methane.bonded_angles()
methane.bonded_dih()

In [8]:
print methane.print_properties()

 n_particles:5 
 n_bonds:4
 n_angles:6
 n_dihedrals:0
 n_impropers:0


In [9]:
methane.particles[1].rsite = 'RH'

In [10]:
methane.particles[2].rsite = 'RH'

In [11]:
methane.find_rsites()
print methane.show_rsites()

rsite:RH[ paticle:atom[1] H (H) index:1 n_bonds:1] 
rsite:RH[ paticle:atom[2] H (H) index:2 n_bonds:1] 



In [12]:
import streamm.structures.buildingblock as bb

In [13]:
ethane = bb.attach(methane,methane,'RH',0,'RH',1,tag='ethane')

In [14]:
oplsaa = streamm.Parameters('oplsaa')

In [15]:
oplsaa.update_units({'energy':'kCalmol'})

In [16]:
CT = streamm.Particletype('CT',unit_conf=oplsaa.unit_conf)
HC = streamm.Particletype('HC',unit_conf=oplsaa.unit_conf)

In [17]:
CT.epsilon = 0.066 # kcal/mol
CT.sigma = 3.5 # Angstroms 
CT.mass = 12.0107
HC.epsilon = 0.03 # kcal/mol
HC.sigma = 2.5 # Angstroms 
HC.mass = 1.00794

In [18]:
oplsaa.add_particletype(CT)
oplsaa.add_particletype(HC)

In [19]:
C_H = streamm.Bondtype('CT','HC',unit_conf=oplsaa.unit_conf)
C_H.setharmonic(1.080,367.0)
C_C = streamm.Bondtype('CT','CT',unit_conf=oplsaa.unit_conf)
C_C.setharmonic(1.080,367.0)

In [20]:
oplsaa.add_bondtype(C_H)
oplsaa.add_bondtype(C_C)

In [21]:
H_C_H = streamm.Angletype('HC','CT','HC',unit_conf=oplsaa.unit_conf)
H_C_H.setharmonic(110.7,37.50)
H_C_C = streamm.Angletype('HC','CT','CT',unit_conf=oplsaa.unit_conf)
H_C_C.setharmonic(90.7,60.50)

In [22]:
oplsaa.add_angletype(H_C_H)
oplsaa.add_angletype(H_C_C)

In [39]:
for pk,p in ethane.particles.iteritems():
    if( p.symbol == 'C' ):
        p.paramkey = 'CT'
    elif( p.symbol == 'H' ):
        p.paramkey = 'HC' 
    print p.paramkey ,ethane.bonded_nblist.calc_nnab(pk)



CT 4
HC 1
HC 1
HC 1
CT 4
HC 1
HC 1
HC 1


In [40]:
md_calc = streamm.LAMMPS('ethane_md')

In [41]:
ethane.update_units(md_calc.unit_conf)

In [42]:
oplsaa.update_units(md_calc.unit_conf)

In [43]:
md_calc.strucC = ethane

In [44]:
md_calc.paramC = oplsaa

In [46]:
md_calc.set_ffparam()

In [47]:
md_calc.write_data()